In [45]:
# Import dependencies
import pandas as pd
import numpy as np
import json
import re

import warnings
warnings.filterwarnings("ignore")

In [8]:

def clean_movie(movie):
    """This function will take the each movie will combine alternate titles and change column names."""
    movie = dict(movie)
    alt_titles = {}
#     Combine alternate titles into one list alt_titles    
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                    'Hangul','Hebrew','Hepburn','Japanese','Literally',
                    'Mandarin','McCune-Reischauer','Original title','Polish',
                    'Revised Romanization','Romanized','Russian',
                    'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    def change_column_name(old_name, new_name):
        """This function will replace the values of old name in new name and pop the old name from list"""
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)

    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [9]:
# Reading the datasets
df_movies = pd.read_csv(r'Resources/movies_metadata.csv')
df_movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [10]:
df_ratings = pd.read_csv(r'Resources/movies_ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [11]:
df_ratings.shape

(100004, 4)

In [12]:
df_ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [13]:
df_ratings['timestamp'] = pd.to_datetime(df_ratings['timestamp'], unit = 's')

In [14]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,2009-12-14 02:52:24
1,1,1029,3.0,2009-12-14 02:52:59
2,1,1061,3.0,2009-12-14 02:53:02
3,1,1129,2.0,2009-12-14 02:53:05
4,1,1172,4.0,2009-12-14 02:53:25


In [15]:
df_movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [16]:
df_movies.shape

(45466, 24)

In [17]:
percent = df_movies.isnull().sum()*100/len(df_movies)
total = df_movies.isnull().sum()
movies_null = pd.DataFrame({'Total':total, 'Percent':percent})
movies_null

,Total,Percent
adult,0,0.000000
belongs_to_collection,40972,90.115691
budget,0,0.000000
genres,0,0.000000
homepage,37684,82.883913
id,0,0.000000
imdb_id,17,0.037391
original_language,11,0.024194
original_title,0,0.000000
overview,954,2.098271


In [18]:
# Keeping columns with null values less than 10% only
drop_list = movies_null[movies_null['Percent']>10].index.tolist()
df_movies.drop(drop_list, axis = 1, inplace = True)
df_movies.columns

Index(['adult', 'budget', 'genres', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [19]:
df_movies.dtypes

adult                    object
budget                   object
genres                   object
id                       object
imdb_id                  object
original_language        object
original_title           object
overview                 object
popularity               object
poster_path              object
production_companies     object
production_countries     object
release_date             object
revenue                 float64
runtime                 float64
spoken_languages         object
status                   object
title                    object
video                    object
vote_average            float64
vote_count              float64
dtype: object

In [20]:
df_movies.adult.value_counts()

False                                                                                                                             45454
True                                                                                                                                  9
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
 - Written by Ørnås                                                                                                                   1
Name: adult, dtype: int64

In [21]:
# Keeping rows where adult column is false and dropping adult column
df_movies = df_movies[df_movies.adult == 'False'].drop('adult', axis = 'columns')

In [22]:
df_movies.video.value_counts()

False    45358
True        93
Name: video, dtype: int64

In [23]:
# Keeping rows with video is false
df_movies.video = df_movies.video == 'True'
df_movies.video.value_counts()

False    45454
Name: video, dtype: int64

In [24]:
# Converting columns to numeric types
df_movies.budget = df_movies.budget.astype(int)
df_movies.id = pd.to_numeric(df_movies.id, errors = 'raise')
df_movies.popularity = pd.to_numeric(df_movies.popularity, errors = 'raise')

In [25]:
# Converting column to datetime datatype
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'])

In [26]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45454 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45454 non-null  int32         
 1   genres                45454 non-null  object        
 2   id                    45454 non-null  int64         
 3   imdb_id               45437 non-null  object        
 4   original_language     45443 non-null  object        
 5   original_title        45454 non-null  object        
 6   overview              44500 non-null  object        
 7   popularity            45451 non-null  float64       
 8   poster_path           45068 non-null  object        
 9   production_companies  45451 non-null  object        
 10  production_countries  45451 non-null  object        
 11  release_date          45368 non-null  datetime64[ns]
 12  revenue               45451 non-null  float64       
 13  runtime         

# X_X_X_X_X

In [20]:
with open(r'Resources/wikipedia-movies.json',mode='r') as file:
    movies_wiki_raw = json.load(file)
len(movies_wiki_raw)

7311

In [21]:
movies_wiki = [movie for movie in movies_wiki_raw
                  if ('Director' in movie or 'Directed by' in movie)
                  and 'imdb_link' in movie
                  and  'No. of episodes' not in movie]
len(movies_wiki)

7076

In [31]:
df_wiki = pd.DataFrame(movies_wiki_raw)
df_wiki.sample(2)

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
2837,https://en.wikipedia.org/wiki/Requiem_for_a_Dream,2000.0,https://www.imdb.com/title/tt0180093/,Requiem for a Dream,Darren Aronofsky,"[Eric Watson, Palmer West]","[Hubert Selby Jr., Darren Aronofsky]",NaN,"[Requiem for a Dream, by Hubert Selby Jr.]","[Ellen Burstyn, Jared Leto, Jennifer Connelly,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2898,https://en.wikipedia.org/wiki/Wonder_Boys_(film),2000.0,https://www.imdb.com/title/tt0185014/,Wonder Boys,Curtis Hanson,"[Curtis Hanson, Scott Rudin]",Steve Kloves,NaN,"[Wonder Boys, by, Michael Chabon]","[Michael Douglas, Tobey Maguire, Frances McDor...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Combining all the alternative titles
sorted(df_wiki.columns.tolist())

['Actor control',
 'Adaptation by',
 'Alias',
 'Alma mater',
 'Also known as',
 'Animation by',
 'Arabic',
 'Area',
 'Area served',
 'Artist(s)',
 'Attraction type',
 'Audio format',
 'Author',
 'Based on',
 'Biographical data',
 'Bopomofo',
 'Born',
 'Box office',
 'Budget',
 'Camera setup',
 'Cantonese',
 'Characters',
 'Children',
 'Chinese',
 'Cinematography',
 'Closing date',
 'Color process',
 'Comics',
 'Composer(s)',
 'Coordinates',
 'Country',
 'Country of origin',
 'Cover artist',
 'Created by',
 'Date premiered',
 'Designer(s)',
 'Developed by',
 'Developer(s)',
 'Dewey Decimal',
 'Died',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Divisions',
 'Duration',
 'Edited by',
 'Editor(s)',
 'Ending theme',
 'Engine',
 'Engine(s)',
 'Executive producer(s)',
 'Family',
 'Fate',
 'Film(s)',
 'Followed by',
 'Format(s)',
 'Formerly',
 'Founded',
 'Founder',
 'Founders',
 'French',
 'Full name',
 'Gender',
 'Genre',
 'Genre(s)',
 'Genres',
 'Gwoyeu Romatzyh',
 'Ha

In [33]:
def clean_movie_columns(movie):
    """This function will handle the alternative titles for the columns"""
    movie = dict(movie)
    alternative_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alternative_titles[key] = movie[key]
            movie.pop(key)
    if len(alternative_titles) > 0:
        movie['alternative_titles'] = alternative_titles
    def change_column_name(old_name, new_name):
        """This function will change the the column names."""
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    return movie

In [34]:
cleaned_movies = [clean_movie_columns(movie) for movie in movies_wiki]
df_wiki_movies = pd.DataFrame(cleaned_movies)

In [35]:
df_wiki_movies.head()

,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Running time,...,Followed by,Created by,Preceded by,Suggested by,alternative_titles,Recorded,Venue,Label,Animation by,Color process
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",NaN,Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",NaN,Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",NaN,Russell Boyd,"December 19, 1990",95 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
total_null_values = df_wiki_movies.isnull().sum()
percent_null_values = df_wiki_movies.isnull().sum() *100 / len(df_wiki_movies)
wiki_null = pd.DataFrame({'null_total': total_null_values, 'null_percent': percent_null_values})
wiki_null

,null_total,null_percent
url,0,0.000000
year,0,0.000000
imdb_link,0,0.000000
title,1,0.014132
Based on,4886,69.050311
Starring,184,2.600339
Narrated by,6794,96.014698
Cinematography,698,9.864330
Release date,32,0.452233
Running time,140,1.978519


In [27]:
with open(r'Resources/wikipedia-movies.json',mode='r') as file:
    movies_wiki_raw = json.load(file)
len(movies_wiki_raw)

7311

In [28]:
movies_wiki_raw[:1]

[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/title/tt0098987/',
  'title': 'The Adventures of Ford Fairlane',
  'Directed by': 'Renny Harlin',
  'Produced by': ['Steve Perry', 'Joel Silver'],
  'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
  'Story by': ['David Arnott', 'James Cappe'],
  'Based on': ['Characters', 'by Rex Weiner'],
  'Starring': ['Andrew Dice Clay',
   'Wayne Newton',
   'Priscilla Presley',
   'Lauren Holly',
   'Morris Day',
   'Robert Englund',
   "Ed O'Neill"],
  'Narrated by': 'Andrew "Dice" Clay',
  'Music by': ['Cliff Eidelman', 'Yello'],
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Michael Tronick',
  'Productioncompany ': 'Silver Pictures',
  'Distributed by': '20th Century Fox',
  'Release date': ['July 11, 1990', '(', '1990-07-11', ')'],
  'Running time': '102 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$20 million',


In [29]:
# Using list comprehension to filter out Tv shows from Movies
movies_wiki = [movie for movie in movies_wiki_raw
               if ('Director' in movie or 'Directed by') in movie 
               and 'imdb_link' in movie   
               and 'No. of episodes' not in movie]

In [30]:
movies_wiki[3453]

{'url': 'https://en.wikipedia.org/wiki/Anacondas:_The_Hunt_for_the_Blood_Orchid',
 'year': 2004,
 'imdb_link': 'https://www.imdb.com/title/tt0366174/',
 'title': 'Anacondas:  The Hunt for the Blood Orchid',
 'Directed by': 'Dwight H. Little',
 'Produced by': 'Verna Harrah',
 'Screenplay by': ['John Claflin',
  'Daniel Zelman',
  'Michael Miner',
  'Edward Neumeier'],
 'Story by': ['Hans Bauer', 'Jim Cash', 'Jack Epps Jr.'],
 'Based on': ['Anaconda',
  'by',
  'Hans Bauer',
  'Jim Cash',
  'and',
  'Jack Epps Jr.'],
 'Starring': ['Johnny Messner',
  'KaDee Strickland',
  'Matthew Marsden',
  'Eugene Byrd',
  'Salli Richardson',
  'Karl Yune',
  'Nicholas Gonzalez',
  'Morris Chestnut'],
 'Music by': 'Nerida Tyson-Chew',
 'Cinematography': 'Stephen F. Windon',
 'Edited by': ["Marcus D'Arcy", 'Mark Warner'],
 'Productioncompany ': ['Columbia Pictures',
  'Screen Gems',
  'Middle Fork Productions'],
 'Distributed by': 'Sony Pictures Releasing',
 'Release date': ['August 27, 2004', '(', '20

In [31]:
# Using a list comprehension to clean each movie with function clean_movie
clean_movies = [clean_movie(movie) for movie in movies_wiki]

In [32]:
clean_movies[3453]

{'url': 'https://en.wikipedia.org/wiki/Anacondas:_The_Hunt_for_the_Blood_Orchid',
 'year': 2004,
 'imdb_link': 'https://www.imdb.com/title/tt0366174/',
 'title': 'Anacondas:  The Hunt for the Blood Orchid',
 'Based on': ['Anaconda',
  'by',
  'Hans Bauer',
  'Jim Cash',
  'and',
  'Jack Epps Jr.'],
 'Starring': ['Johnny Messner',
  'KaDee Strickland',
  'Matthew Marsden',
  'Eugene Byrd',
  'Salli Richardson',
  'Karl Yune',
  'Nicholas Gonzalez',
  'Morris Chestnut'],
 'Cinematography': 'Stephen F. Windon',
 'Release date': ['August 27, 2004', '(', '2004-08-27', ')'],
 'Running time': '97 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$20–25 million',
 'Box office': '$71 million',
 'Director': 'Dwight H. Little',
 'Distributor': 'Sony Pictures Releasing',
 'Editor(s)': ["Marcus D'Arcy", 'Mark Warner'],
 'Composer(s)': 'Nerida Tyson-Chew',
 'Producer(s)': 'Verna Harrah',
 'Production company(s)': ['Columbia Pictures',
  'Screen Gems',
  'Middle Fork Producti

In [33]:
movies_wiki_df = pd.DataFrame(clean_movies)

In [34]:
movies_wiki_df['imdb_link'].head()

0    https://www.imdb.com/title/tt0098987/
1    https://www.imdb.com/title/tt0098994/
2    https://www.imdb.com/title/tt0099005/
3    https://www.imdb.com/title/tt0099012/
4    https://www.imdb.com/title/tt0099018/
Name: imdb_link, dtype: object

In [35]:
# Using regexp to extract id from link and drop duplicate rows from id
movies_wiki_df['imdb_id'] = movies_wiki_df['imdb_link'].str.extract(r'(tt\d{7})')
movies_wiki_df.drop_duplicates('imdb_id', inplace = True)

In [36]:
# Using list comprehension to keep only columns with null values less than 10%
wiki_columns_keeping = [column for column in movies_wiki_df.columns 
                         if movies_wiki_df[column].isnull().sum() < len(movies_wiki_df) * 0.9]
movies_wiki_df = movies_wiki_df[wiki_columns_keeping]

In [37]:
box_office = movies_wiki_df['Box office'].dropna()

In [41]:
# Convert box_office to string values using lambda function
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) ==  list else x)

In [42]:
# Using regular expressions to match two types of patterns in box office data
type_one = r'\$\d+\.?\d*\s*[mb]illion'
type_two = r'\$\d{1,3}(?:,\d{3})+'

In [43]:
def parse_dollars(s):
    if type(s) != str:
        return np.nan
    # Matches only the input of type $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
        s = re.sub('\$|\s|[a-zA-Z]', '', s)  # Removes the $ sign and ' million'
        value = float(s) * 10 ** 6  # Convert string to float and multiply by a million
        return value
    # Matches only the input of type $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
        s = re.sub('\$|\s|[a-zA-Z]', '', s)  # Removes the $ sign and ' billion'
        value = float(s) * 10 ** 9  # Convert string to float and multiply by a billion
        return value
    # Matches only the input of type $###.# billion
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
        s = re.sub('\$|,', '', s)  # Removes $ dollar sign and commas
        value = float(s)  # Convert string to float
        return value
    else:
        return np.nan

In [46]:
# Cleaning the box_office column using the parse_dollar function
movies_wiki_df['box_office'] = box_office.str.extract(f'({type_one}|{type_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
movies_wiki_df.drop("Box office", axis=1, inplace = True)

In [47]:
movies_wiki_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Running time,Country,...,Budget,Director,Distributor,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,United States,...,$20 million,Renny Harlin,20th Century Fox,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987,21400000.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,United States,...,$6 million,James Foley,Avenue Pictures,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994,2700000.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,United States,...,$35 million,Roger Spottiswoode,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005,57718089.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,United States,...,$12 million,Woody Allen,Orion Pictures,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012,7331647.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",95 minutes,US,...,$25 million,John Cornell,Paramount Pictures,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018,6939946.0
